In [ ]:
require 'pco_api'

token =  ENV['PCO_API_TOKEN']
secret = ENV['PCO_API_SECRET']

api = PCO::API.new(basic_auth_token: token, basic_auth_secret: secret)

require 'sqlite3'

# Open a database
db = SQLite3::Database.new "songs.db"

nil

In [ ]:
db.execute <<-SQL
drop table songs;
SQL


# Create a table
rows = db.execute <<-SQL
  create table songs (
    title varchar(50),
    id varchar(20),
    ccli int,
    themes text

  );
SQL


nil

## Load songs from PCO

In [ ]:
db.execute 'delete from songs'

offset = 0
per_page = 25
fetch_more = true

while fetch_more

  response = api.services.v2.songs.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |song|
    attrs = song['attributes']
    title = attrs['title']
    themes = attrs['themes']
    ccli = attrs['ccli_number']
    id = song['id']
    
    db.execute "insert into songs values ( ?, ?, ?, ?)", [ title, id, ccli, themes]
  end
  
  fetch_more = data.length == per_page
  offset = offset + per_page

end

# Find a few rows
db.execute( "select count(*) as count from songs" ) do |row|
  p row[0]
end

nil

## Load Arrangements from PCO

In [ ]:
db.execute <<-SQL
drop table arrangements;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table arrangements (
    song_id varchar(20),
    id varchar(20),
    name varchar(50),
    bpm int,
    meter varchar(10),
    notes text,
    length int,
    sequence text,
    has_chords varchar(10),
    has_chor_chart varchar(10)
  );
SQL

nil

In [ ]:
# Load arrangements from PCO
db.execute( "select id from songs" ) do |row|
  song_id = row[0]
  offset = 0
  per_page = 100

  response = api.services.v2.songs[song_id.to_i].arrangements.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |arrangement|
    id = arrangement['id']
    attrs = arrangement['attributes']

    bpm = attrs["bpm"]
    has_chord_chart = attrs['has_chord_chart']
    has_chords = attrs["has_chords"]
    length = attrs['length']
    meter = attrs['meter']
    name = attrs['name']
    notes = attrs['notes']
    sequence = attrs['sequence']
    
    db.execute "insert into arrangements values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)",
      [song_id, id, name, bpm, meter, notes, length, sequence.to_json, has_chords.to_s, has_chord_chart.to_s]
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select count(*) as count from arrangements" ) do |row|
  p row[0]
end

nil

# Load Song Keys

In [ ]:
db.execute <<-SQL
drop table keys;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table keys (
    song_id varchar(20),
    arrangement_id varchar(20),
    id varchar(20),
    name varchar(50),
    alternate_keys varchar(50),
    ending_key varchar(50),
    starting_key varchar(50),
    starting_minor int,
    ending_minor int
  );
SQL

nil

In [ ]:
# Load arrangements from PCO
db.execute( "select song_id, id from arrangements" ) do |row|
  song_id = row[0]
  arrangement_id = row[1]
  offset = 0
  per_page = 100
  
  response = api.services.v2.songs[song_id.to_i].arrangements[arrangement_id.to_i].keys.get(offset: offset, per_page: per_page)

  data = response["data"]

  data.each do |key|
    id = key['id']
    attrs = key['attributes']
    
    name = attrs["name"]
    alternate_keys = attrs['alternate_keys']
    ending_key = attrs["ending_key"]
    starting_key = attrs['starting_key']
    starting_minor = attrs['starting_minor']
    ending_minor = attrs['ending_minor']

    
    db.execute "insert into keys values ( ?, ?, ?, ?, ?, ?, ?, ?, ?)",
      [song_id, arrangement_id, id, name, alternate_keys.to_json, ending_key, starting_key, (starting_minor ? 1 : 0), (ending_minor ? 1 : 0)]
  end
end


nil

## Load Schedule for songs from PCO
(This actually isn't helpful because it's only the future schedule, not historical)

In [ ]:
db.execute <<-SQL
drop table song_schedules;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table song_schedules (
    id varchar(20),
    song_id varchar(20),
    arrangement_id varchar(20),
    key_id varchar(20),
    plan_id varchar(20),
    service_type_id varchar(20),
    item_id varchar(20)
  );
SQL

nil

In [ ]:
db.execute( "delete from song_schedules" )

# Load arrangements from PCO
db.execute( "select id from songs" ) do |row|
  
  song_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.songs[song_id.to_i].song_schedules.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |song_schedule|
      id = song_schedule['id']
      attrs = song_schedule['attributes']
      rel = song_schedule['relationships']

      arrangement_id = rel['arrangement']['data']['id']
      key_id = rel['key']['data']['id']
      plan_id = rel['plan']['data']['id']
      service_type_id = rel['service_type']['data']['id']
      item_id = rel['item']['data']['id']
      
      db.execute "insert into song_schedules values ( ?, ?, ?, ?, ?, ?, ?)",
        [id, song_id, arrangement_id, key_id, plan_id, service_type_id, item_id]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select * from song_schedules" ) do |row|
  p row
end

nil

## Load Service Types

In [ ]:
db.execute <<-SQL
  drop table service_types;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table service_types (
    id varchar(20),
    name varchar(50)
  );
SQL

nil

In [ ]:
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |service_type|
      id = service_type['id']
      attrs = service_type['attributes']
      
      name = attrs['name']
      
      db.execute "insert into service_types values ( ?, ?)", [id, name]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end

In [ ]:
# Find a few rows
db.execute( "select * from service_types" ) do |row|
  p row
end

nil

## Load Plans

In [ ]:
db.execute <<-SQL
  drop table plans;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table plans (
    id varchar(20),
    service_type_id varchar(20),
    dates text
  );
SQL

nil

In [ ]:
require 'date'
db.execute( "delete from plans" )

# Load service types from PCO
db.execute( "select id from service_types" ) do |row|
  
  service_type_id = row[0]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
        
      dates = attrs['dates'] == 'No dates' ? nil : Date.strptime(attrs['dates'], '%B %d, %Y')
      short_dates = attrs['short_dates']
      
      db.execute "insert into plans values ( ?, ?, ?)", [id, service_type_id, dates.to_s]
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end

nil

In [ ]:
# Find a few rows
db.execute( "select * from plans" ) do |row|
  p row
end


## Plan Items

In [ ]:
db.execute <<-SQL
drop table plan_items;
SQL

# Create table for arrangements
db.execute <<-SQL
  create table plan_items (
    id varchar(20),
    plan_id varchar(20),
    title text,
    description text,
    item_type text,
    length int,
    service_position text,
    service_sequence int,
    custom_arrangement_sequence text,
    song_id varchar(20),
    arrangement_id varchar(20),
    key_id varchar(20)
  );
SQL

nil

In [ ]:
#db.execute( "delete from plan_items" )

# Load service types from PCO
db.execute( "select id, service_type_id from plans" ) do |row|
  plan_id = row[0]
  service_type_id = row[1]
  offset = 0
  per_page = 100
  fetch_more = true

  while fetch_more

    response = api.services.v2.service_types[service_type_id.to_i].plans[plan_id.to_i].items.get(offset: offset, per_page: per_page)

    data = response["data"]

    data.each do |plan|
      id = plan['id']
      attrs = plan['attributes']
      rel = plan['relationships']
      
      custom_arrangement_sequence = attrs['custom_arrangement_sequence']
      description = attrs['description']
      item_type = attrs['item_type']
      length = attrs['length']
      sequence = attrs['sequence']
      service_position = attrs['service_position']
      title = attrs['title']
      
      song_id = rel.dig('song', 'data', 'id')
      arrangement_id = rel.dig('arrangement', 'data', 'id')
      key_id = rel.dig('key', 'data', 'id')
            
      values = [id, plan_id, title, description, item_type, length, service_position, sequence, custom_arrangement_sequence.to_json, song_id, arrangement_id, key_id]
      db.execute "insert into plan_items values ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)", values
    end
    
    fetch_more = data.length == per_page
    offset = offset + per_page
  end
end


nil

In [ ]:
# Find a few rows
db.execute( "select count(*) from plan_items" ) do |row|
  p row
end


In [ ]:
def date_of_next(day)
  date  = Date.parse(day)
  delta = date > Date.today ? 0 : 7
  date + delta
end

sunday_songs = []

db.execute( <<-SQL 
  select
    PI.title,
    A.bpm,
    A.meter,
    K.starting_key
  from 
  plan_items PI
  LEFT JOIN plans P on P.id = PI.plan_id
  LEFT JOIN service_types ST on ST.id = P.service_type_id
  LEFT JOIN songs S on S.id = PI.song_id
  LEFT JOIN arrangements A on A.id = PI.arrangement_id
  LEFT JOIN keys K on K.id = PI.key_id
  WHERE
    ST.name = "TCBR Sunday Morning Service"
    AND 
    P.dates = "#{date_of_next('Sunday')}"
    AND
    PI.item_type = 'song'
  SQL
  ) do |row|
  p row
 
  sunday_songs << row
end

nil

In [ ]:
require 'zlib'
require 'nokogiri'

project_xml = Zlib::GzipReader.open('Template.als') {|gz|
  project_xml = gz.read
}

project_data  = Nokogiri::XML(project_xml)

scenes = project_data.xpath("/Ableton/LiveSet/SceneNames/Scene")
p scenes

sunday_songs.each_with_index do |song, index|
  new_value = "(#{song[3]}) #{song[0]}; #{song[1]} BPM; #{song[2]}"
  p index
  scenes[index][:Value] = new_value
end

#new_xml = Ox.dump(project_data)
#new_xml = project_xml
new_xml = project_data.to_xml
sunday = date_of_next("Sunday")

File.open("TCBR_Worship_#{sunday.strftime('%Y_%m_%d')}.xml", 'w') { |file| file.write(new_xml) }

Zlib::GzipWriter.open("TCBR_Worship_#{sunday.strftime('%Y_%m_%d')}.als") do |gz|
  gz.write new_xml
end


## Plot frequency diagram

In [ ]:
db.execute(<<-SQL
  SELECT
    s.title,
    count(p.dates) as count
  FROM
    plan_items pi
    JOIN songs s ON s.id = pi.song_id
    JOIN plans p on p.id = pi.plan_id
  GROUP BY
    s.title
  ORDER BY
    count(p.dates) desc
SQL
  ) do |row|
    p row
  end

nil